In [1]:
import tokenizers
from transformers import BertTokenizer

In [2]:
tokenizer=BertTokenizer(vocab_file='./train_data/vocab.txt', tokenize_chinese_chars=False)

In [4]:
from transformers import AlbertConfig, AlbertModel, AlbertForMaskedLM

# Initializing an ALBERT style configuration
albert_configuration = AlbertConfig(
    vocab_size=len(tokenizer),
    embedding_size=128,
    hidden_size=128,
    num_attention_heads=4,
    num_hidden_layers = 2,
    intermediate_size=128,
    hidden_dropout_prob=0.1,
    attention_probs_dropout_prob=0.1,
)

# Initializing a model from the ALBERT-base style configuration
model = AlbertForMaskedLM(albert_configuration)

# Accessing the model configuration
configuration = model.config
configuration

AlbertConfig {
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "hidden_act": "gelu_new",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 128,
  "initializer_range": 0.02,
  "inner_group_num": 1,
  "intermediate_size": 128,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "albert",
  "num_attention_heads": 4,
  "num_hidden_groups": 1,
  "num_hidden_layers": 2,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.6.1",
  "type_vocab_size": 2,
  "vocab_size": 25122
}

In [5]:
import os
import csv
from transformers import  BertTokenizer, WEIGHTS_NAME,TrainingArguments
import tokenizers
import torch
from datasets import load_dataset, Dataset 
from transformers import (
    CONFIG_MAPPING,
    MODEL_FOR_MASKED_LM_MAPPING,
    AutoConfig,
    AutoModelForMaskedLM,
    AutoTokenizer,
    DataCollatorForLanguageModeling,
    HfArgumentParser,
    Trainer,
    TrainingArguments,
    set_seed,
    LineByLineTextDataset
)


# 通过LineByLineTextDataset接口 加载数据 #长度设置为128, # 这里file_path于本文第一部分的语料格式一致
train_dataset=LineByLineTextDataset(tokenizer=tokenizer,file_path='./train_data/xiaohuangji50w_nofenci.txt', block_size=16) 
# MLM模型的数据DataCollator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)
# 训练参数


C:\Users\RuiLi\.conda\envs\pytorch_gpu\lib\site-packages\transformers\data\datasets\language_modeling.py:120: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [6]:
for data in train_dataset:
    print(data)
    break

{'input_ids': tensor([ 2, 64,  3])}


In [7]:
pretrain_batch_size=256
num_train_epochs=100
training_args = TrainingArguments(
    output_dir='./outputs/', overwrite_output_dir=True, num_train_epochs=num_train_epochs, learning_rate=1e-3,
    per_device_train_batch_size=pretrain_batch_size, save_total_limit=5, logging_strategy ='epoch', logging_steps=100)# save_steps=10000
# 通过Trainer接口训练模型

trainer = Trainer(
    model=model, args=training_args, data_collator=data_collator, train_dataset=train_dataset)

# 开始训练
trainer.train(resume_from_checkpoint = None)
# trainer.train(True)
trainer.save_model('./outputs/')

  0%|          | 210/331600 [00:15<6:31:10, 14.12it/s]

KeyboardInterrupt: 

In [ ]:
1e-5 , 3.1